# Project Proposal

In [28]:
# Loading Libraries, Remember to run this cell!
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

### Data Analysis

In [33]:
# Loading in the data
player_stats <- read_csv("https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS") %>% # Loading data using a url
    select("Age":"Peak Elo Rating", -Country, -Wikipedia, -Name, -Seasons, -Active, -"Favorite Surface", -"Best Elo Rank", "Peak Elo Rating") # Remove unnecessary columns
player_stats

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.



Age,Plays,Current Rank,Best Rank,Backhand,Prize Money,Height,Turned Pro,Current Elo Rank,Peak Elo Rating
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
26 (25-04-1993),Right-handed,378 (97),363 (04-11-2019),NA,NA,NA,NA,NA,NA
18 (22-12-2001),Left-handed,326 (119),316 (14-10-2019),Two-handed,"$59,040",NA,NA,NA,NA
32 (03-11-1987),Right-handed,178 (280),44 (14-01-2013),Two-handed,"US$3,261,567",185 cm,2005,144 (1764),1886 (06-02-2012)
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
23 (14-03-1996),Left-handed,495 (60),342 (05-08-2019),NA,NA,NA,NA,NA,NA
24 (17-05-1995),NA,419 (81),419 (20-01-2020),NA,NA,NA,NA,NA,NA
22 (26-03-1997),Left-handed,451 (69),408 (24-12-2018),Two-handed,"$61,984",NA,NA,NA,NA
